**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [52]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip install sk-video 
import skvideo.io
import cv2
import json
import tensorflow as tf

from keras.models import Sequential,model_from_json
from keras import regularizers as reg
from keras.layers.core import Dense, Dropout
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act proposes an action given the current state and all the previous knowledge accumulated.
With probability $ \epsilon $ it returns a random action and with probability $ 1 - \epsilon $ it returns an action from the current policy. $ \epsilon $ Allows to explore the different possible actions in order to obtain more information. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=40 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- The *position* array allows to store the position of the rat and the possible cells the rat can reach. A cell is set to -1 if the rat can't go to and 0 otherwise..
- the *board* array stores the possible rewards on our game grid over time. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):

        action = np.random.randint(low=0, high=4) # We pick a random action among: 0, 1, 2 and 3
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
 
        win = 0
        lose = 0
 
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
    
            # # Apply the reinforcement strategy
            if prefix != 'random':
                loss = agent.reinforce(prev_state, state,  action, reward, game_over)
 
        # Save as a mp4
        if ((e + 1) % 10 == 0) or (e == epochs - 1):
            env.draw(prefix+str(e+1))
        if prefix != 'random':
            agent.save()

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [59]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random' + str(epochs_test) + '.mp4'))

Win/lose count 9.0/16.0. Average score (-7.0)
Win/lose count 8.0/8.0. Average score (-3.5)
Win/lose count 10.5/11.0. Average score (-2.5)
Win/lose count 10.5/7.0. Average score (-1.0)
Win/lose count 12.5/9.0. Average score (-0.1)
Win/lose count 11.5/19.0. Average score (-1.3333333333333333)
Win/lose count 8.5/16.0. Average score (-2.2142857142857144)
Win/lose count 7.5/5.0. Average score (-1.625)
Win/lose count 8.0/6.0. Average score (-1.2222222222222223)
Win/lose count 9.0/10.0. Average score (-1.2)
Final score: -1.2


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




- 1)
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + E_{(s',a')\sim p(.|s,a)}\left(\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)
\end{align*}
$$

- 2)
 Using the optmal greedy policy, which is deterministic; $\pi^*(s') = arg\max_{a'} Q^{*}(s',a')$, we have:$$
\begin{align*}
Q^*(s,a)&= E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s'\sim p(.|s,a)\,and\,a'\sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}
$$

- 3)
From the previous result, we have:  $$Q^*(s,a)=E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)$$

By puting everything in the expectation, we have:
$$E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*
}(s',a') - Q^{*
}(s,a)\right) = 0$$

Then minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will allow us to find $Q^*$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # We append m to memory if the size of memory doesn't exceed max_memory
        if len(self.memory) == self.max_memory:
            self.memory.pop(0)
            self.memory.append(m)
        else:
            self.memory.append(m)

    def random_access(self):
        
        idx = np.random.randint(low=0, high=len(self.memory)) # We choose a random index
        return self.memory[idx]
        

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1 % 10 == 0) or (e+1==epoch):
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        self.epsilon = epsilon
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):

        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):

            # we sample a random transition
            s, ns, a, r, game_over = self.memory.random_access()
            input_states[i], target_q[i] = s, self.model.predict(s[None])
            # We have to deal with the case when we are loosing
            if game_over:
                target_q[i, a] = r
            else:
                target_q[i, a] =  r + self.discount*np.max(self.model.predict(ns[None]), axis=1)
            
        
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):

        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(36, input_shape=(5*5*self.n_state, ), kernel_regularizer=reg.l2(0.001)))
        model.add(Activation('relu'))
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4), "mse")
        self.model = model
        
        

In [63]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train' + str(epochs_train) + '.mp4'))

Epoch 001/040 | Loss 0.0497 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/040 | Loss 0.0421 | Win/lose count 0.5/2.0 (-1.5)
Epoch 003/040 | Loss 0.0421 | Win/lose count 2.0/5.0 (-3.0)
Epoch 004/040 | Loss 0.0547 | Win/lose count 5.0/6.0 (-1.0)
Epoch 005/040 | Loss 0.0335 | Win/lose count 1.5/1.0 (0.5)
Epoch 006/040 | Loss 0.0347 | Win/lose count 5.0/2.0 (3.0)
Epoch 007/040 | Loss 0.0351 | Win/lose count 1.0/1.0 (0.0)
Epoch 008/040 | Loss 0.0284 | Win/lose count 2.5/5.0 (-2.5)
Epoch 009/040 | Loss 0.0269 | Win/lose count 4.5/1.0 (3.5)
Epoch 010/040 | Loss 0.0777 | Win/lose count 5.5/6.0 (-0.5)
Epoch 011/040 | Loss 0.0279 | Win/lose count 2.5/3.0 (-0.5)
Epoch 012/040 | Loss 0.0280 | Win/lose count 3.0/6.0 (-3.0)
Epoch 013/040 | Loss 0.0223 | Win/lose count 6.0/4.0 (2.0)
Epoch 014/040 | Loss 0.0600 | Win/lose count 3.0/5.0 (-2.0)
Epoch 015/040 | Loss 0.0194 | Win/lose count 1.5/0 (1.5)
Epoch 016/040 | Loss 0.0182 | Win/lose count 3.0/4.0 (-1.0)
Epoch 017/040 | Loss 0.0220 | Win/lose count 7.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(filters=60, kernel_size=3, activation='relu', kernel_regularizer=reg.l2(0.001)))
        model.add(Conv2D(filters=30, kernel_size=3, activation='relu', kernel_regularizer=reg.l2(0.001)))

        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4), "mse")
        self.model = model

In [65]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train' + str(epochs_train) + '.mp4'))

Epoch 001/040 | Loss 0.0468 | Win/lose count 3.0/10.0 (-7.0)
Epoch 002/040 | Loss 0.0440 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/040 | Loss 0.0371 | Win/lose count 3.5/0 (3.5)
Epoch 004/040 | Loss 0.0340 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/040 | Loss 0.0401 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/040 | Loss 0.0437 | Win/lose count 4.0/5.0 (-1.0)
Epoch 007/040 | Loss 0.0275 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/040 | Loss 0.0297 | Win/lose count 3.0/3.0 (0.0)
Epoch 009/040 | Loss 0.0288 | Win/lose count 4.0/3.0 (1.0)
Epoch 010/040 | Loss 0.0258 | Win/lose count 1.5/4.0 (-2.5)
Epoch 011/040 | Loss 0.0229 | Win/lose count 5.5/4.0 (1.5)
Epoch 012/040 | Loss 0.0265 | Win/lose count 2.5/6.0 (-3.5)
Epoch 013/040 | Loss 0.0270 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/040 | Loss 0.0220 | Win/lose count 5.0/7.0 (-2.0)
Epoch 015/040 | Loss 0.0192 | Win/lose count 4.5/0 (4.5)
Epoch 016/040 | Loss 0.0195 | Win/lose count 2.5/3.0 (-0.5)
Epoch 017/040 | Loss 0.0182 | Win/lose count 6.0/1.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [66]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/1.0. Average score (4.0)
Win/lose count 6.0/3.0. Average score (3.5)
Win/lose count 20.5/4.0. Average score (7.833333333333333)
Win/lose count 9.0/6.0. Average score (6.625)
Win/lose count 13.5/4.0. Average score (7.2)
Win/lose count 6.0/4.0. Average score (6.333333333333333)
Win/lose count 9.5/2.0. Average score (6.5)
Win/lose count 5.0/3.0. Average score (5.9375)
Win/lose count 13.5/0. Average score (6.777777777777778)
Win/lose count 13.0/1.0. Average score (7.3)
Final score: 7.3
Test of the FC
Win/lose count 4.5/0. Average score (4.5)
Win/lose count 8.0/18.0. Average score (-2.75)
Win/lose count 5.0/7.0. Average score (-2.5)
Win/lose count 8.0/7.0. Average score (-1.625)
Win/lose count 4.5/4.0. Average score (-1.2)
Win/lose count 3.0/0. Average score (-0.5)
Win/lose count 3.0/6.0. Average score (-0.8571428571428571)
Win/lose count 6.5/3.0. Average score (-0.3125)
Win/lose count 4.5/0. Average score (0.2222222222222222)
Win/lose count 12.0/5.0. Aver

In [67]:
HTML(display_videos('cnn_test' + str(epochs_test) + '.mp4'))

In [68]:
HTML(display_videos('fc_test' + str(epochs_test) + '.mp4'))

- Both agents get easily stuck when there is no more "cheese" (reward) near the agent. 
- We don't explore enough the map so we tend to exploit what we already know, that explains why the agent gets stuck.

- Increasing the temperature leads to a grid with a higher number of "cheese cells" compared to poisonous cells. The agent explores more and has a highest score since there are a lot more positive rewards on his path than negative ones.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



- We will make $ \epsilon $ decrease from a factor $ \gamma $ (between 0 and 1) at each epoch in order to explore a lot at the beginning to gain knowledge and then to exploit the optimal policy we find.

In [0]:
def train_explore(agent,env,epoch, gamma=0.8, prefix=''):
    
    # Number of won games
    score = 0
    loss = 0
    
    min_eps = 0.1 # minimum value of epsilon

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # New epsilon
        if e >= 1:
            new_eps = max(min_eps, agent.epsilon*gamma)
            agent.set_epsilon(new_eps)
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
            
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        
        # Save as a mp4
        if (e+1 % 10 == 0) or (e == epoch - 1):
            env.draw(prefix + str(e+1))
    
        # Update stats
        score += win-lose
    
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | epsilon: {}"
              .format(e+1, epoch, loss, win, lose, win-lose, agent.epsilon))
        agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')
        

class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size,
                                                   max_time=max_time,
                                                   temperature=temperature)
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

    def draw(self, e):
        super(EnvironmentExploring, self).draw(e)
    
    def get_frame(self, t):
        super(EnvironmentExploring, self).get_frame(t)
    
    def act(self, action, train=False):

        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
             reward = -self.malus_position[self.x, self.y]
        
        reward += self.board[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over
    
    def reset(self):
        
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state

    
# ## use those samples of code:
# #In train explore:
# state, reward, game_over = env.act(action, train=True)

# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [70]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, gamma=0.8, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore' + str(epochs_train) + '.mp4'))

Epoch 001/040 | Loss 0.0585 | Win/lose count 12.0/14.0 (-2.0) | epsilon: 1
Epoch 002/040 | Loss 0.0495 | Win/lose count 8.5/10.0 (-1.5) | epsilon: 0.8
Epoch 003/040 | Loss 0.0436 | Win/lose count 9.5/11.0 (-1.5) | epsilon: 0.6400000000000001
Epoch 004/040 | Loss 0.1040 | Win/lose count 14.5/10.0 (4.5) | epsilon: 0.5120000000000001
Epoch 005/040 | Loss 0.0382 | Win/lose count 12.5/7.0 (5.5) | epsilon: 0.40960000000000013
Epoch 006/040 | Loss 0.0920 | Win/lose count 18.0/12.0 (6.0) | epsilon: 0.32768000000000014
Epoch 007/040 | Loss 0.0307 | Win/lose count 6.5/4.0 (2.5) | epsilon: 0.2621440000000001
Epoch 008/040 | Loss 0.0353 | Win/lose count 12.0/5.0 (7.0) | epsilon: 0.2097152000000001
Epoch 009/040 | Loss 0.0343 | Win/lose count 15.0/3.0 (12.0) | epsilon: 0.1677721600000001
Epoch 010/040 | Loss 0.1284 | Win/lose count 15.5/2.0 (13.5) | epsilon: 0.13421772800000006
Epoch 011/040 | Loss 0.0246 | Win/lose count 9.5/1.0 (8.5) | epsilon: 0.10737418240000006
Epoch 012/040 | Loss 0.0271 | Wi

In [71]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore' + str(epochs_test) + '.mp4'))

Win/lose count 11.0/3.0. Average score (8.0)
Win/lose count 7.0/1.0. Average score (7.0)
Win/lose count 24.5/2.0. Average score (12.166666666666666)
Win/lose count 7.0/2.0. Average score (10.375)
Win/lose count 14.0/0. Average score (11.1)
Win/lose count 19.0/1.0. Average score (12.25)
Win/lose count 12.0/0. Average score (12.214285714285714)
Win/lose count 11.0/2.0. Average score (11.8125)
Win/lose count 13.0/2.0. Average score (11.722222222222221)
Win/lose count 10.0/3.0. Average score (11.25)
Final score: 11.25


- We observe an important difference of performance compared to the previous case without an adaptative epsilon (For the same number of epochs).

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***